In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

try:
    import scikeras
except ImportError:
    !python -m pip install scikeras

from scikeras.wrappers import KerasClassifier, KerasRegressor

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
    
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando o modelo a ser otimizado

Para realizar a otimização com as classes `GridSearchCV` ou `RandomizedSearchCV` da biblioteca SciKit-Learn, precisamos envolver nossos modelos em objetos que imitem regressores do Scikit-Learn. 

O primeiro passo é criar uma função que irá construir e compilar um modelo.

Esta função cria um modelo `Sequencial` para regressão, com a dimensão da entrada e o número de camadas ocultas e neurônios, e o compila usando um otimizador `SGD` configurado com a taxa de aprendizado fornecida.

In [4]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

### Regressor Keras

Em seguida, criamos um objeto da classe `KerasRegressor` baseado na função `build_model()`.

O objeto da classe `KerasRegressor` é um wrapper em torno do modelo construído usando a função `build_model()`. 

Como não especificamos nenhum hiperparâmetro ao criá-lo, ele usará apenas os hiperparâmetros padrão que definimos em na função `build_model()`.

In [5]:
keras_reg = KerasRegressor(build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)

Agora podemos usar esse objeto como um regressor do Scikit-Learn: podemos treiná-lo usando seu método `fit()`, avaliá-lo usando seu método `score()` e usá-lo para fazer predições usando seu método `predict()`.

In [6]:
keras_reg.fit(X_train, y_train, 
              epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]
             )

Epoch 1/100
363/363 [==============================] - 2s 4ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 1s 3ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4060 - val_l

363/363 [==============================] - 1s 3ms/step - loss: 0.3384 - val_loss: 0.3411
Epoch 82/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3371 - val_loss: 0.3245
Epoch 83/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3368 - val_loss: 0.3269
Epoch 84/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3362 - val_loss: 0.4066
Epoch 85/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3373 - val_loss: 0.3460
Epoch 86/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3360 - val_loss: 0.3190
Epoch 87/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3356 - val_loss: 0.3259
Epoch 88/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3351 - val_loss: 0.3329
Epoch 89/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3349 - val_loss: 0.3221
Epoch 90/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3344 - val_loss: 

KerasRegressor(
	model=<function build_model at 0x000002C6C2F9EE50>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_hidden=1
	n_neurons=30
	learning_rate=0.003
)

In [7]:
mse_test = keras_reg.score(X_test, y_test)

print('MSE test:', mse_test)

162/162 [==============================] - 1s 2ms/step
MSE test: 0.7474111269910106


In [8]:
X_new = X_test[:3]

y_pred = keras_reg.predict(X_new)

print('Predictions:', y_pred)

1/1 [==============================] - 0s 41ms/step
Predictions: [0.6034199 1.5396053 4.1040044]


### Otimização hiperparamétrica
 

Agora vamos treinar vários modelos com diferentes valores para os hiperparâmetros e ver qual conjunto de valores tem o melhor desempenho no conjunto de validação. 


Como existem muitos hiperparâmetros a serem testados, é preferível usar uma pesquisa aleatória (`RandomSearchCV`) em vez de uma pesquisa em grade.


Vamos tentar explorar o número de camadas ocultas, o número de neurônios e a taxa de aprendizado.

In [ ]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)

rnd_search_cv.fit(X_train, y_train, 
                  epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)]
                 )

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 2s 6ms/step - loss: 3.3071 - val_loss: 1.3735
Epoch 2/100
242/242 [==============================] - 1s 4ms/step - loss: 0.9782 - val_loss: 0.6881
Epoch 3/100
242/242 [==============================] - 1s 4ms/step - loss: 0.6186 - val_loss: 0.5887
Epoch 4/100
242/242 [==============================] - 1s 4ms/step - loss: 0.5504 - val_loss: 0.5466
Epoch 5/100
242/242 [==============================] - 1s 4ms/step - loss: 0.5363 - val_loss: 0.5360
Epoch 6/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5307 - val_loss: 0.7261
Epoch 7/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5322 - val_loss: 0.6728
Epoch 8/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5311 - val_loss: 0.6939
Epoch 9/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5326 - val_loss: 0.5351
Epoch 10/100
242/242 [========

Epoch 81/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5246 - val_loss: 0.8111
Epoch 82/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5307 - val_loss: 0.5353
Epoch 83/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5280 - val_loss: 0.4954
Epoch 84/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5272 - val_loss: 0.5445
Epoch 85/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5270 - val_loss: 0.6084
Epoch 86/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5272 - val_loss: 0.6462
Epoch 87/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5290 - val_loss: 0.4923
Epoch 88/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5276 - val_loss: 0.5009
Epoch 89/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5257 - val_loss: 0.6698
Epoch 90/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5275

242/242 [==============================] - 1s 2ms/step - loss: 0.4986 - val_loss: 20.6228
Epoch 60/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4986 - val_loss: 20.4560
Epoch 61/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4984 - val_loss: 20.5028
Epoch 62/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4984 - val_loss: 20.4879
Epoch 63/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4986 - val_loss: 20.5138
Epoch 64/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4984 - val_loss: 20.5009
Epoch 65/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4983 - val_loss: 20.6074
Epoch 66/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4985 - val_loss: 20.6702
Epoch 67/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4984 - val_loss: 20.6235
Epoch 68/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4984 - v

242/242 [==============================] - 1s 2ms/step - loss: 0.5332 - val_loss: 0.4935
Epoch 38/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5298 - val_loss: 0.7451
Epoch 39/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5344 - val_loss: 0.5824
Epoch 40/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5319 - val_loss: 0.6109
Epoch 41/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5295 - val_loss: 0.8136
Epoch 42/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5340 - val_loss: 0.6256
Epoch 43/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5316 - val_loss: 0.5192
Epoch 44/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5300 - val_loss: 0.7006
Epoch 45/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5302 - val_loss: 0.7742
Epoch 46/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5315 - val_loss: 

242/242 [==============================] - 1s 3ms/step - loss: 19965354.0000 - val_loss: 5623153152.0000
Epoch 16/100
242/242 [==============================] - 1s 3ms/step - loss: 100124888.0000 - val_loss: 22019987456.0000
Epoch 17/100
242/242 [==============================] - 1s 3ms/step - loss: 490005120.0000 - val_loss: 86582329344.0000
Epoch 18/100
242/242 [==============================] - 1s 3ms/step - loss: 2112389888.0000 - val_loss: 340066598912.0000
Epoch 19/100
242/242 [==============================] - 1s 3ms/step - loss: 8082988544.0000 - val_loss: 1331716816896.0000
Epoch 20/100
242/242 [==============================] - 1s 3ms/step - loss: 31350368256.0000 - val_loss: 5207425548288.0000
Epoch 21/100
242/242 [==============================] - 1s 3ms/step - loss: 58745643008.0000 - val_loss: 20485045747712.0000
Epoch 22/100
242/242 [==============================] - 1s 3ms/step - loss: 456048214016.0000 - val_loss: 80743405453312.0000
Epoch 23/100
242/242 [=============

...

242/242 [==============================] - 1s 2ms/step - loss: 182229.5781 - val_loss: 24290910.0000
Epoch 20/100
242/242 [==============================] - 1s 2ms/step - loss: 412970.0000 - val_loss: 55526112.0000
Epoch 21/100
242/242 [==============================] - 1s 2ms/step - loss: 579729.6250 - val_loss: 127407696.0000
Epoch 22/100
242/242 [==============================] - 1s 2ms/step - loss: 2090171.0000 - val_loss: 293123488.0000
Epoch 23/100
242/242 [==============================] - 1s 3ms/step - loss: 4541476.0000 - val_loss: 670447232.0000
Epoch 24/100
242/242 [==============================] - 1s 3ms/step - loss: 8376887.5000 - val_loss: 1531425024.0000
Epoch 25/100
242/242 [==============================] - 1s 3ms/step - loss: 19211898.0000 - val_loss: 3498238976.0000
Epoch 26/100
242/242 [==============================] - 1s 3ms/step - loss: 26036824.0000 - val_loss: 8019315200.0000
Epoch 27/100
242/242 [==============================] - 1s 3ms/step - loss: 149462928

Epoch 80/100
242/242 [==============================] - 1s 2ms/step - loss: 1984540129695015006451531776.0000 - val_loss: 394981452710311258589176528896.0000
Epoch 81/100
242/242 [==============================] - 1s 3ms/step - loss: 7036826040027145192094040064.0000 - val_loss: 914779887266145698693037162496.0000
Epoch 82/100
242/242 [==============================] - 1s 2ms/step - loss: 9956998753518299496415494144.0000 - val_loss: 2110391882387222688212497465344.0000
Epoch 83/100
242/242 [==============================] - 1s 2ms/step - loss: 16833100821188407389778345984.0000 - val_loss: 4848498549333317934009794166784.0000
Epoch 84/100
242/242 [==============================] - 1s 2ms/step - loss: 52690007094376829866557308928.0000 - val_loss: 11007002414985063802652122415104.0000
Epoch 85/100
242/242 [==============================] - 1s 2ms/step - loss: 146915805817691836213857615872.0000 - val_loss: 25223241071383875275804404350976.0000
Epoch 86/100
242/242 [====================

Epoch 48/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5014 - val_loss: 18.7524
Epoch 49/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5009 - val_loss: 20.3974
Epoch 50/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5009 - val_loss: 20.5749
Epoch 51/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5005 - val_loss: 20.0806
Epoch 52/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5008 - val_loss: 21.0705
Epoch 53/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5011 - val_loss: 19.4328
Epoch 54/100
242/242 [==============================] - 1s 4ms/step - loss: 0.5000 - val_loss: 20.2420
Epoch 55/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5006 - val_loss: 21.1054
Epoch 56/100
242/242 [==============================] - 1s 4ms/step - loss: 0.5010 - val_loss: 20.8038
Epoch 57/100
242/242 [==============================] - 1s 4ms/step - los

242/242 [==============================] - 1s 2ms/step - loss: 2780.5720 - val_loss: 724655.0625
Epoch 26/100
242/242 [==============================] - 1s 3ms/step - loss: 3028.7935 - val_loss: 1038362.6875
Epoch 27/100
242/242 [==============================] - 1s 4ms/step - loss: 21503.0371 - val_loss: 1508422.0000
Epoch 28/100
242/242 [==============================] - 1s 4ms/step - loss: 12912.3926 - val_loss: 2159898.7500
Epoch 29/100
242/242 [==============================] - 1s 3ms/step - loss: 36257.6250 - val_loss: 3109370.7500
Epoch 30/100
242/242 [==============================] - 1s 2ms/step - loss: 26729.7617 - val_loss: 4462223.5000
Epoch 31/100
242/242 [==============================] - 1s 3ms/step - loss: 41369.8984 - val_loss: 6305890.0000
Epoch 32/100
242/242 [==============================] - 1s 3ms/step - loss: 113821.4688 - val_loss: 8987188.0000
Epoch 33/100
242/242 [==============================] - 1s 2ms/step - loss: 158755.2969 - val_loss: 12746436.0000
Epoch

Epoch 93/100
242/242 [==============================] - 1s 2ms/step - loss: 263396989599744.0000 - val_loss: 24995610151092224.0000
Epoch 94/100
242/242 [==============================] - 1s 3ms/step - loss: 209541220270080.0000 - val_loss: 35551293292412928.0000
Epoch 95/100
242/242 [==============================] - 1s 3ms/step - loss: 207973859196928.0000 - val_loss: 50831294430445568.0000
Epoch 96/100
242/242 [==============================] - 1s 3ms/step - loss: 447838588764160.0000 - val_loss: 72193297824612352.0000
Epoch 97/100
242/242 [==============================] - 1s 3ms/step - loss: 1215254285516800.0000 - val_loss: 102582630985236480.0000
Epoch 98/100
242/242 [==============================] - 1s 2ms/step - loss: 464560305733632.0000 - val_loss: 149243155688980480.0000
Epoch 99/100
242/242 [==============================] - 1s 2ms/step - loss: 1451498022109184.0000 - val_loss: 211782208845774848.0000
Epoch 100/100
121/121 [==============================] - 0s 1ms/step
[C

A busca pode durar muitas horas dependendo do hardware, do tamanho do conjunto de dados, da complexidade do modelo e do valor de `n_iter` e `cv`. 

Quando terminar, podemos acessar os melhores hiperparâmetros encontrados, a melhor pontuação e o modelo treinado como mostrado abaixo.

In [ ]:
# Melhores hiperparâmetros.
rnd_search_cv.best_params_

# Score do melhor conjunto de hiperparâmetros.
rnd_search_cv.best_score_

# Melhor modelo.
model = rnd_search_cv.best_estimator_.model

#### Conclusões

+ A pesquisa aleatória funciona bem (encontra o melhor conjunto de hiperparâmetros) para muitos problemas simples. 


+ No entanto, quando o treinamento é lento (por exemplo, para problemas mais complexos com conjuntos de dados maiores), essa abordagem explorará apenas uma pequena parte do espaço de hiperparâmetros e pode não encontrar o melhor conjunto de valores. 


+ Podemos aliviar parcialmente esse problema auxiliando o processo de pesquisa manualmente: 
    + Primeiro executamos uma pesquisa aleatória rápida usando grandes intervalos de valores de hiperparâmetros, 
    + Na sequência, executamos outra pesquisa usando intervalos menores de valores centrados nos melhores encontrados durante a primeira execução e assim por diante. 


+ Esperamos que isso ajude a encontrar um bom conjunto de hiperparâmetros. No entanto, isso consome muito tempo.